In [ ]:
!git clone https://github.com/meettyj/RecipeRec.git

Cloning into 'RecipeRec'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 2), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 26.97 KiB | 13.49 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [1]:
!cp -r /content/drive/MyDrive/data_mlp_project/ /content/data

In [2]:
!nvidia-smi

Mon Mar 10 19:01:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)
import platform
print(platform.platform())
print(platform.system(), platform.release())

2.5.1+cu124
12.4
Linux-6.1.85+-x86_64-with-glibc2.35
Linux 6.1.85+


In [4]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.

In [1]:
!pip install torchfile

  Preparing metadata (setup.py) ... done
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5693 sha256=72c83c2c90c0cf41b715cfeac996e5d6446ee53a6012f2a2edf1702ad4645b16
  Stored in directory: /root/.cache/pip/wheels/2d/0d/6c/cc20d113923479bc37343fc2268b9b369a1b25a4cb97296b3a
Successfully built torchfile


In [2]:
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 7.7 MB/s eta 0:00:00


In [3]:
import json
import pickle
import re
import nltk
from collections import Counter
import pandas as pd
import random
import heapq
import csv
from tqdm import tqdm
import os
import numpy as np
import time
import math
import lmdb
import gensim
import heapq
os.environ["DGL_GRAPHBOLT_DISABLE"] = "1"

import dgl
import dgl.nn as dglnn
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.functional import edge_softmax
import torchfile
from torch.nn import init
import dgl.function as fn
from dgl.utils import expand_as_pair
from dgl.nn import EdgeWeightNorm
from dgl.distributed import DistDataLoader
from dgl.distributed.dist_dataloader import EdgeCollator

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/usr/local/lib/python3.11/dist-packages/dgl/graphbolt/__init__.py:114: GBWarning: 
An experimental feature for CUDA allocations is turned on for better allocation
pattern resulting in better memory usage for minibatch GNN training workloads.
See https://pytorch.org/docs/stable/notes/cuda.html#optimizing-memory-usage-with-pytorch-cuda-alloc-conf,
and set the environment variable `PYTORCH_CUDA_ALLOC_CONF=expandable_segments:False`
if you want to disable it and set it True to acknowledge and disable the warning.

  gb_warning(WARNING_STR_TO_BE_SHOWN)


In [4]:
device = ("cuda:0" if torch.cuda.is_available() else "cpu")
print('device: ', device)

device:  cuda:0


In [7]:
# parameters
Ks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # top@K performance
n_test_negs = 100 # number of negative recipes for each test user
dataset_folder = '/content/data'

## Build graph

In [8]:
def get_graph():
    print('generating graph ...')
    edge_src, edge_dst, r_i_edge_weight = torch.load(dataset_folder+'/edge_r2i_src_dst_weight.pt')
    recipe_edge_src, recipe_edge_dst, recipe_edge_weight = torch.load(dataset_folder+'/edge_r2r_src_and_dst_and_weight.pt')
    ingre_edge_src, ingre_edge_dst, ingre_edge_weight = torch.load(dataset_folder+'/edge_i2i_src_and_dst_and_weight.pt')
    all_u2r_src_dst_weight, train_u2r_src_dst_weight, val_u2r_src_dst_weight, test_u2r_src_dst_weight = torch.load(dataset_folder+'/all_train_val_test_edge_u_rate_r_src_and_dst_and_weight.pt')
    u_rate_r_edge_src, u_rate_r_edge_dst, u_rate_r_edge_weight = all_u2r_src_dst_weight

    # nodes and edges
    graph = dgl.heterograph({
        ('recipe', 'r-i', 'ingredient'): (edge_src, edge_dst),
        ('ingredient', 'i-r', 'recipe'): (edge_dst, edge_src),
        ('recipe', 'r-r', 'recipe'): (recipe_edge_src, recipe_edge_dst),
        ('ingredient', 'i-i', 'ingredient'): (ingre_edge_src, ingre_edge_dst),
        ('user', 'u-r', 'recipe'): (u_rate_r_edge_src, u_rate_r_edge_dst),
        ('recipe', 'r-u', 'user'): (u_rate_r_edge_dst, u_rate_r_edge_src)
    })

    metapaths = {
    'recipe-ingredient-recipe': [('recipe', 'r-i', 'ingredient'), ('ingredient', 'i-r', 'recipe')],
    'user-recipe-user': [('user', 'u-r', 'recipe'), ('recipe', 'r-u', 'user')],
    'recipe-recipe': [('recipe', 'r-r', 'recipe')],
    'ingredient-ingredient': [('ingredient', 'i-i', 'ingredient')]
    }
    graph.metapaths = metapaths

    # edge weight
    graph.edges['r-i'].data['weight'] = torch.FloatTensor(r_i_edge_weight)
    graph.edges['i-r'].data['weight'] = torch.FloatTensor(r_i_edge_weight)
    graph.edges['r-r'].data['weight'] = torch.FloatTensor(recipe_edge_weight)
    graph.edges['i-i'].data['weight'] = torch.FloatTensor(ingre_edge_weight)
    graph.edges['u-r'].data['weight'] = torch.FloatTensor(u_rate_r_edge_weight)
    graph.edges['r-u'].data['weight'] = torch.FloatTensor(u_rate_r_edge_weight)

    # node features
    recipe_nodes_avg_instruction_features = torch.load(dataset_folder+'/recipe_nodes_avg_instruction_features.pt')
    ingredient_nodes_nutrient_features_minus1 = torch.load(dataset_folder+'/ingredient_nodes_nutrient_features.pt')
    graph.nodes['recipe'].data['avg_instr_feature'] = recipe_nodes_avg_instruction_features
    graph.nodes['ingredient'].data['nutrient_feature'] = ingredient_nodes_nutrient_features_minus1
    graph.nodes['user'].data['random_feature'] = torch.nn.init.xavier_normal_(torch.ones(7959, 300))
    graph.nodes['recipe'].data['random_feature'] = torch.nn.init.xavier_normal_(torch.ones(68794, 1024))

    return graph

graph = get_graph()
print('graph: ', graph)

generating graph ...


<ipython-input-8-e236fe1c4574>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  edge_src, edge_dst, r_i_edge_weight = torch.load(dataset_folder+'/edge_r2i_src_dst_weight.pt'

graph:  Graph(num_nodes={'ingredient': 8847, 'recipe': 68794, 'user': 7959},
      num_edges={('ingredient', 'i-i', 'ingredient'): 146188, ('ingredient', 'i-r', 'recipe'): 463485, ('recipe', 'r-i', 'ingredient'): 463485, ('recipe', 'r-r', 'recipe'): 647146, ('recipe', 'r-u', 'user'): 135353, ('user', 'u-r', 'recipe'): 135353},
      metagraph=[('ingredient', 'ingredient', 'i-i'), ('ingredient', 'recipe', 'i-r'), ('recipe', 'ingredient', 'r-i'), ('recipe', 'recipe', 'r-r'), ('recipe', 'user', 'r-u'), ('user', 'recipe', 'u-r')])


## Split train/val/test

In [9]:
all_src_dst_weight, train_src_dst_weight, val_src_dst_weight, test_src_dst_weight = torch.load(dataset_folder+'/all_train_val_test_edge_u_rate_r_src_and_dst_and_weight.pt')
all_src, all_dst, all_weight = all_src_dst_weight
train_src, train_dst, train_weight = train_src_dst_weight
val_src, val_dst, val_weight = val_src_dst_weight
test_src, test_dst, test_weight = test_src_dst_weight

train_eids = graph.edge_ids(train_src, train_dst, etype='u-r')
val_eids = graph.edge_ids(val_src, val_dst, etype='u-r')
test_eids = graph.edge_ids(test_src, test_dst, etype='u-r')
val_eids_r2u = graph.edge_ids(val_dst, val_src, etype='r-u')
test_eids_r2u = graph.edge_ids(test_dst, test_src, etype='r-u')
print('length of all_src: ', len(all_src))
print('length of train_eids: ', len(train_eids))
print('length of val_eids: ', len(val_eids))
print('length of test_eids: ', len(test_eids))

length of all_src:  135353
length of train_eids:  119435
length of val_eids:  7959
length of test_eids:  7959


<ipython-input-9-63b0a4d482de>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  all_src_dst_weight, train_src_dst_weight, val_src_dst_weight, test_src_dst_weight = torch.loa

In [10]:
# get train_graph and val_graph
train_graph = graph.clone()
train_graph.remove_edges(torch.cat([val_eids, test_eids]), etype='u-r')
train_graph.remove_edges(torch.cat([val_eids_r2u, test_eids_r2u]), etype='r-u')
print('training graph: ')
print(train_graph)
print()

val_graph = graph.clone()
val_graph.remove_edges(test_eids, etype='u-r')
val_graph.remove_edges(test_eids, etype='r-u')
print('val graph: ')
print(val_graph)

for g in [train_graph, val_graph, graph]:
    if not hasattr(g, '_use_graphbolt'):
        g._use_graphbolt = False

training graph: 
Graph(num_nodes={'ingredient': 8847, 'recipe': 68794, 'user': 7959},
      num_edges={('ingredient', 'i-i', 'ingredient'): 146188, ('ingredient', 'i-r', 'recipe'): 463485, ('recipe', 'r-i', 'ingredient'): 463485, ('recipe', 'r-r', 'recipe'): 647146, ('recipe', 'r-u', 'user'): 119435, ('user', 'u-r', 'recipe'): 119435},
      metagraph=[('ingredient', 'ingredient', 'i-i'), ('ingredient', 'recipe', 'i-r'), ('recipe', 'ingredient', 'r-i'), ('recipe', 'recipe', 'r-r'), ('recipe', 'user', 'r-u'), ('user', 'recipe', 'u-r')])

val graph: 
Graph(num_nodes={'ingredient': 8847, 'recipe': 68794, 'user': 7959},
      num_edges={('ingredient', 'i-i', 'ingredient'): 146188, ('ingredient', 'i-r', 'recipe'): 463485, ('recipe', 'r-i', 'ingredient'): 463485, ('recipe', 'r-r', 'recipe'): 647146, ('recipe', 'r-u', 'user'): 127394, ('user', 'u-r', 'recipe'): 127394},
      metagraph=[('ingredient', 'ingredient', 'i-i'), ('ingredient', 'recipe', 'i-r'), ('recipe', 'ingredient', 'r-i'), ('re

In [11]:
# edge dataloaders
sampler = dgl.dataloading.MultiLayerNeighborSampler([20, 20])
neg_sampler = dgl.dataloading.negative_sampler.Uniform(5)

class test_NegativeSampler(object):
    def __init__(self, g, k):
        # get the negatives
        self.user2negs_100_dict = {}
        filename = dataset_folder+'/test_negatives_100.txt'
        with open(filename, "r") as f:
            lines = f.readlines()
            for line in tqdm(lines):
                if line == None or line == "":
                    continue
                line = line[:-1] # remove \n
                user = int(line.split('\t')[0].split(',')[0][1:])
                negs = [int(neg) for neg in line.split('\t')[1:]]
                self.user2negs_100_dict[user] = negs

        self.k = k

    def __call__(self, g, eids_dict):
        result_dict = {}
        for etype, eids in eids_dict.items():
            src, _ = g.find_edges(eids, etype=etype)
            dst = []
            for each_src in src:
                dst.extend(self.user2negs_100_dict[int(each_src)][:self.k])
            dst = torch.tensor(dst)
            src = src.repeat_interleave(self.k)
            result_dict[etype] = (src, dst)
        return result_dict

test_neg_sampler = test_NegativeSampler(graph, n_test_negs)
test_train_neg_sampler = test_NegativeSampler(graph, n_test_negs)

train_collator = EdgeCollator(
    train_graph, {'u-r': train_graph.edge_ids(train_src, train_dst, etype='u-r')}, sampler,
    exclude='reverse_types',
    reverse_etypes={'u-r': 'r-u', 'r-u': 'u-r'},
    negative_sampler=neg_sampler)
val_collator = EdgeCollator(
    val_graph, {'u-r': val_graph.edge_ids(val_src, val_dst, etype='u-r')}, sampler,
    exclude='reverse_types',
    reverse_etypes={'u-r': 'r-u', 'r-u': 'u-r'},
    negative_sampler=neg_sampler)
test_collator = EdgeCollator(
    graph, {('user', 'u-r', 'recipe'): test_eids}, sampler,
    exclude='reverse_types',
    reverse_etypes={'u-r': 'r-u', 'r-u': 'u-r'},
    negative_sampler=test_neg_sampler)

train_edgeloader = torch.utils.data.DataLoader(
    train_collator.dataset, collate_fn=train_collator.collate,
    batch_size=1024, shuffle=True, drop_last=False, num_workers=0)
val_edgeloader = torch.utils.data.DataLoader(
    val_collator.dataset, collate_fn=val_collator.collate,
    batch_size=128, shuffle=False, drop_last=False, num_workers=0)
test_edgeloader = torch.utils.data.DataLoader(
    test_collator.dataset, collate_fn=test_collator.collate,
    batch_size=128, shuffle=False, drop_last=False, num_workers=0)

print('# of batches in train_edgeloader: ', len(train_edgeloader))
print('# of batches in val_edgeloader: ', len(val_edgeloader))
print('# of batches in test_edgeloader: ', len(test_edgeloader))
print()

for input_nodes, pos_pair_graph, neg_pair_graph, blocks in train_edgeloader:
    print('blocks: ', blocks)
    break

100%|██████████| 7959/7959 [00:00<00:00, 28873.32it/s]


# of batches in train_edgeloader:  117
# of batches in val_edgeloader:  63
# of batches in test_edgeloader:  63

blocks:  [Block(num_src_nodes={'ingredient': 6530, 'recipe': 61784, 'user': 5805},
      num_dst_nodes={'ingredient': 3680, 'recipe': 27127, 'user': 3050},
      num_edges={('ingredient', 'i-i', 'ingredient'): 47753, ('ingredient', 'i-r', 'recipe'): 178069, ('recipe', 'r-i', 'ingredient'): 50523, ('recipe', 'r-r', 'recipe'): 252475, ('recipe', 'r-u', 'user'): 35573, ('user', 'u-r', 'recipe'): 52401},
      metagraph=[('ingredient', 'ingredient', 'i-i'), ('ingredient', 'recipe', 'i-r'), ('recipe', 'ingredient', 'r-i'), ('recipe', 'recipe', 'r-r'), ('recipe', 'user', 'r-u'), ('user', 'recipe', 'u-r')]), Block(num_src_nodes={'ingredient': 3680, 'recipe': 27127, 'user': 3050},
      num_dst_nodes={'ingredient': 0, 'recipe': 5867, 'user': 682},
      num_edges={('ingredient', 'i-i', 'ingredient'): 0, ('ingredient', 'i-r', 'recipe'): 39700, ('recipe', 'r-i', 'ingredient'): 0, ('re

## Set Transformer

In [ ]:
# get ingre neighbors for each recipe nodes
def get_recipe2ingreNeighbor_dict():
    max_length = 33
    out = {}
    neighbor_list = []
    ingre_length_list = []
    total_length_index_list = []
    total_ingre_neighbor_list = []
    total_length_index = 0
    total_length_index_list.append(total_length_index)
    for recipeNodeID in tqdm(range(graph.number_of_nodes('recipe'))):
        _, succs = graph.out_edges(recipeNodeID, etype='r-i')
        succs_list = list(set(succs.tolist()))
        total_ingre_neighbor_list.extend(succs_list)
        cur_length = len(succs_list)
        ingre_length_list.append(cur_length)

        total_length_index += cur_length
        total_length_index_list.append(total_length_index)
        while len(succs_list) < max_length:
            succs_list.append(77733)
        neighbor_list.append(succs_list)

    ingre_neighbor_tensor = torch.tensor(neighbor_list)
    ingre_length_tensor = torch.tensor(ingre_length_list)
    total_ingre_neighbor_tensor = torch.tensor(total_ingre_neighbor_list)
    return ingre_neighbor_tensor, ingre_length_tensor, total_length_index_list, total_ingre_neighbor_tensor

ingre_neighbor_tensor, ingre_length_tensor, total_length_index_list, total_ingre_neighbor_tensor = get_recipe2ingreNeighbor_dict()
print('ingre_neighbor_tensor: ', ingre_neighbor_tensor.shape)
print('ingre_length_tensor: ', ingre_length_tensor.shape)
print('total_length_index_list: ', len(total_length_index_list))
print('total_ingre_neighbor_tensor: ', total_ingre_neighbor_tensor.shape)

def find(tensor, values):
    return torch.nonzero(tensor[..., None] == values)

# example of find()
# a = torch.tensor([0, 10, 20, 30])
# b = torch.tensor([[ 0, 30, 20,  10, 77733],[ 0, 30, 20,  10, 77733]])
# find(b, a)[:, 2]

100%|██████████| 68794/68794 [00:12<00:00, 5299.40it/s]


ingre_neighbor_tensor:  torch.Size([68794, 33])
ingre_length_tensor:  torch.Size([68794])
total_length_index_list:  68795
total_ingre_neighbor_tensor:  torch.Size([454941])


In [ ]:
def get_ingredient_neighbors_all_embeddings(blocks, output_nodes, secondToLast_ingre):
    ingreNodeIDs = blocks[1].srcdata['_ID']['ingredient']
    recipeNodeIDs = output_nodes
    batch_ingre_neighbors = ingre_neighbor_tensor[recipeNodeIDs.cpu()].to(device)
    batch_ingre_length = ingre_length_tensor[recipeNodeIDs.cpu()].to(device)
    valid_batch_ingre_neighbors = find(batch_ingre_neighbors, ingreNodeIDs)[:, 2]

    # based on valid_batch_ingre_neighbors each row index
    _, valid_batch_ingre_length = torch.unique(find(batch_ingre_neighbors, ingreNodeIDs)[:, 0], return_counts=True)
    batch_sum_ingre_length = np.cumsum(valid_batch_ingre_length.cpu())

    total_ingre_emb = None
    for i in range(len(recipeNodeIDs)):
        if i == 0:
            recipeNode_ingres = valid_batch_ingre_neighbors[0:batch_sum_ingre_length[i]]
            a = secondToLast_ingre[recipeNode_ingres]
        else:
            recipeNode_ingres = valid_batch_ingre_neighbors[batch_sum_ingre_length[i-1]:batch_sum_ingre_length[i]]
            a = secondToLast_ingre[recipeNode_ingres]

        # all ingre instead of average
        a_rows = a.shape[0]
        a_columns = a.shape[1]
        max_rows = 5
        if a_rows < max_rows:
            a = torch.cat([a, torch.zeros(max_rows-a_rows, a_columns).cuda()])
        else:
            a = a[:max_rows, :]

        if total_ingre_emb == None:
            total_ingre_emb = a.unsqueeze(0)
        else:
            total_ingre_emb = torch.cat([total_ingre_emb,a.unsqueeze(0)], dim = 0)
            if torch.isnan(total_ingre_emb).any():
                print('Error!')

    return total_ingre_emb

In [ ]:
class Attention(nn.Module):
    """Scaled Dot-Product Attention."""

    def __init__(self, temperature):
        super().__init__()

        self.temperature = temperature
        self.softmax = nn.Softmax(dim=2)

    def forward(self, queries, keys, values):
        """
        It is equivariant to permutations
        of the batch dimension (`b`).

        It is equivariant to permutations of the
        second dimension of the queries (`n`).

        It is invariant to permutations of the
        second dimension of keys and values (`m`).

        Arguments:
            queries: a float tensor with shape [b, n, d].
            keys: a float tensor with shape [b, m, d].
            values: a float tensor with shape [b, m, d'].
        Returns:
            a float tensor with shape [b, n, d'].
        """

        attention = torch.bmm(queries, keys.transpose(1, 2))
        attention = self.softmax(attention / self.temperature)
        # it has shape [b, n, m]

        return torch.bmm(attention, values)


class MultiheadAttention(nn.Module):

    def __init__(self, d, h):
        """
        Arguments:
            d: an integer, dimension of queries and values.
                It is assumed that input and
                output dimensions are the same.
            h: an integer, number of heads.
        """
        super().__init__()

        assert d % h == 0
        self.h = h

        # everything is projected to this dimension
        p = d // h

        self.project_queries = nn.Linear(d, d)
        self.project_keys = nn.Linear(d, d)
        self.project_values = nn.Linear(d, d)
        self.concatenation = nn.Linear(d, d)
        self.attention = Attention(temperature=p**0.5)

    def forward(self, queries, keys, values):
        """
        Arguments:
            queries: a float tensor with shape [b, n, d].
            keys: a float tensor with shape [b, m, d].
            values: a float tensor with shape [b, m, d].
        Returns:
            a float tensor with shape [b, n, d].
        """

        h = self.h
        b, n, d = queries.size()
        _, m, _ = keys.size()
        p = d // h

        queries = self.project_queries(queries)  # shape [b, n, d]
        keys = self.project_keys(keys)  # shape [b, m, d]
        values = self.project_values(values)  # shape [b, m, d]

        queries = queries.view(b, n, h, p)
        keys = keys.view(b, m, h, p)
        values = values.view(b, m, h, p)

        queries = queries.permute(2, 0, 1, 3).contiguous().view(h * b, n, p)
        keys = keys.permute(2, 0, 1, 3).contiguous().view(h * b, m, p)
        values = values.permute(2, 0, 1, 3).contiguous().view(h * b, m, p)

        output = self.attention(queries, keys, values)  # shape [h * b, n, p]
        output = output.view(h, b, n, p)
        output = output.permute(1, 2, 0, 3).contiguous().view(b, n, d)
        output = self.concatenation(output)  # shape [b, n, d]

        return output

class RFF(nn.Module):
    """
    Row-wise FeedForward layers.
    """
    def __init__(self, d):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(d, d), nn.ReLU(inplace=True),
            nn.Linear(d, d), nn.ReLU(inplace=True),
            nn.Linear(d, d), nn.ReLU(inplace=True)
        )

    def forward(self, x):
        """
        Arguments:
            x: a float tensor with shape [b, n, d].
        Returns:
            a float tensor with shape [b, n, d].
        """
        return self.layers(x)

class MultiheadAttentionBlock(nn.Module):

    def __init__(self, d, h, rff):
        """
        Arguments:
            d: an integer, input dimension.
            h: an integer, number of heads.
            rff: a module, row-wise feedforward layers.
                It takes a float tensor with shape [b, n, d] and
                returns a float tensor with the same shape.
        """
        super().__init__()

        self.multihead = MultiheadAttention(d, h)
        self.layer_norm1 = nn.LayerNorm(d)
        self.layer_norm2 = nn.LayerNorm(d)
        self.rff = rff

    def forward(self, x, y):
        """
        It is equivariant to permutations of the
        second dimension of tensor x (`n`).

        It is invariant to permutations of the
        second dimension of tensor y (`m`).

        Arguments:
            x: a float tensor with shape [b, n, d].
            y: a float tensor with shape [b, m, d].
        Returns:
            a float tensor with shape [b, n, d].
        """
        h = self.layer_norm1(x + self.multihead(x, y, y))
        return self.layer_norm2(h + self.rff(h))

class SetAttentionBlock(nn.Module):

    def __init__(self, d, h, rff):
        super().__init__()
        self.mab = MultiheadAttentionBlock(d, h, rff)

    def forward(self, x):
        """
        Arguments:
            x: a float tensor with shape [b, n, d].
        Returns:
            a float tensor with shape [b, n, d].
        """
        return self.mab(x, x)

class InducedSetAttentionBlock(nn.Module):

    def __init__(self, d, m, h, rff1, rff2):
        """
        Arguments:
            d: an integer, input dimension.
            m: an integer, number of inducing points.
            h: an integer, number of heads.
            rff1, rff2: modules, row-wise feedforward layers.
                It takes a float tensor with shape [b, n, d] and
                returns a float tensor with the same shape.
        """
        super().__init__()
        self.mab1 = MultiheadAttentionBlock(d, h, rff1)
        self.mab2 = MultiheadAttentionBlock(d, h, rff2)
        self.inducing_points = nn.Parameter(torch.randn(1, m, d))

    def forward(self, x):
        """
        Arguments:
            x: a float tensor with shape [b, n, d].
        Returns:
            a float tensor with shape [b, n, d].
        """
        b = x.size(0)
        p = self.inducing_points
        p = p.repeat([b, 1, 1])  # shape [b, m, d]
        h = self.mab1(p, x)  # shape [b, m, d]
        return self.mab2(x, h)

class PoolingMultiheadAttention(nn.Module):

    def __init__(self, d, k, h, rff):
        """
        Arguments:
            d: an integer, input dimension.
            k: an integer, number of seed vectors.
            h: an integer, number of heads.
            rff: a module, row-wise feedforward layers.
                It takes a float tensor with shape [b, n, d] and
                returns a float tensor with the same shape.
        """
        super().__init__()
        self.mab = MultiheadAttentionBlock(d, h, rff)
        self.seed_vectors = nn.Parameter(torch.randn(1, k, d))

    def forward(self, z):
        """
        Arguments:
            z: a float tensor with shape [b, n, d].
        Returns:
            a float tensor with shape [b, k, d].
        """
        b = z.size(0)
        s = self.seed_vectors
        s = s.repeat([b, 1, 1])  # random seed vector: shape [b, k, d]

        output = self.mab(s, z)
        # print('PoolingMultiheadAttention', output.shape)

        return output

In [ ]:
# Set transformer for ingredient representation
class SetTransformer(nn.Module):
    def __init__(self):
        """
        Arguments:
            in_dimension: an integer.  # 2
            out_dimension: an integer. # 5 * K
        """
        super(SetTransformer, self).__init__()
        in_dimension = 46 # 300
        out_dimension = 128 # 600

        d = in_dimension
        m = 46  # number of inducing points
        h = 2  # 4 # number of heads
        k = 4  # number of seed vectors

        self.encoder = nn.Sequential(
            InducedSetAttentionBlock(d, m, h, RFF(d), RFF(d)),
            InducedSetAttentionBlock(d, m, h, RFF(d), RFF(d))
        )
        self.decoder = nn.Sequential(
            PoolingMultiheadAttention(d, k, h, RFF(d)),
            SetAttentionBlock(d, h, RFF(d))
        )

        self.decoder_2 = nn.Sequential(
            PoolingMultiheadAttention(d, k, h, RFF(d))
        )
        self.decoder_3 = nn.Sequential(
            SetAttentionBlock(d, h, RFF(d))
        )

        self.predictor = nn.Sequential(
            nn.Linear(k * d, out_dimension),
            nn.ReLU()
        )

        self.dropout = nn.Dropout(0.1)


    def forward(self, x):
        """
        Arguments:
            x: a float tensor with shape [batch, n, in_dimension].
        Returns:
            a float tensor with shape [batch, out_dimension].
        """
        x = self.encoder(x) # x = self.encoder(cut_x) # shape [batch, batch_max_len, d]
        x = self.dropout(x)
        x = self.decoder(x)  # shape [batch, k, d]

        b, k, d = x.shape
        x = x.view(b, k * d)

        y = self.predictor(x)
        return y

## GNN

In [ ]:
class custom_HeteroGraphConv(nn.Module):
    def __init__(self, mods, aggregate='sum'):
        super(custom_HeteroGraphConv, self).__init__()
        self.mods = nn.ModuleDict(mods)
        # Do not break if graph has 0-in-degree nodes.
        # Because there is no general rule to add self-loop for heterograph.
        for _, v in self.mods.items():
            set_allow_zero_in_degree_fn = getattr(v, 'set_allow_zero_in_degree', None)
            if callable(set_allow_zero_in_degree_fn):
                set_allow_zero_in_degree_fn(True)
        if isinstance(aggregate, str):
            self.agg_fn = get_aggregate_fn(aggregate)
        else:
            self.agg_fn = aggregate

    def forward(self, g, inputs, mod_args=None, mod_kwargs=None):
        if mod_args is None:
            mod_args = {}
        if mod_kwargs is None:
            mod_kwargs = {}
        outputs = {nty : [] for nty in g.dsttypes}
        if isinstance(inputs, tuple) or g.is_block:
            if isinstance(inputs, tuple):
                src_inputs, dst_inputs = inputs
            else:
                src_inputs = inputs
                dst_inputs = {k: v[:g.number_of_dst_nodes(k)] for k, v in inputs.items()}

            for stype, etype, dtype in g.canonical_etypes:
                rel_graph = g[stype, etype, dtype]
                if rel_graph.number_of_edges() == 0:
                    continue
                if stype not in src_inputs or dtype not in dst_inputs:
                    continue
                dstdata = self.mods[etype](
                    rel_graph,
                    (src_inputs[stype], dst_inputs[dtype], mod_kwargs),
                    *mod_args.get(etype, ())
                    )
                outputs[dtype].append(dstdata)
        else:
            for stype, etype, dtype in g.canonical_etypes:
                rel_graph = g[stype, etype, dtype]
                if rel_graph.number_of_edges() == 0:
                    continue
                if stype not in inputs:
                    continue
                dstdata = self.mods[etype](
                    rel_graph,
                    (inputs[stype], inputs[dtype], mod_kwargs),
                    *mod_args.get(etype, ())
                    )
                outputs[dtype].append(dstdata)
        rsts = {}
        for nty, alist in outputs.items():
            if len(alist) != 0:
                rsts[nty] = self.agg_fn(alist, nty)
        return rsts

def _max_reduce_func(inputs, dim):
    return torch.max(inputs, dim=dim)[0]

def _min_reduce_func(inputs, dim):
    return torch.min(inputs, dim=dim)[0]

def _sum_reduce_func(inputs, dim):
    return torch.sum(inputs, dim=dim)

def _mean_reduce_func(inputs, dim):
    return torch.mean(inputs, dim=dim)

def _stack_agg_func(inputs, dsttype):
    if len(inputs) == 0:
        return None
    return torch.stack(inputs, dim=1)

def _agg_func(inputs, dsttype, fn):
    if len(inputs) == 0:
        return None
    stacked = torch.stack(inputs, dim=0)
    return fn(stacked, dim=0)

def get_aggregate_fn(agg):
    if agg == 'sum':
        fn = _sum_reduce_func
    elif agg == 'max':
        fn = _max_reduce_func
    elif agg == 'min':
        fn = _min_reduce_func
    elif agg == 'mean':
        fn = _mean_reduce_func
    elif agg == 'stack':
        fn = None  # will not be called
    else:
        raise DGLError('Invalid cross type aggregator. Must be one of '
                       '"sum", "max", "min", "mean" or "stack". But got "%s"' % agg)
    if agg == 'stack':
        return _stack_agg_func
    else:
        return partial(_agg_func, fn=fn)

In [ ]:
class ScorePredictor(nn.Module):
    def forward(self, edge_subgraph, x):
        with edge_subgraph.local_scope():
            edge_subgraph.ndata['x'] = x
            edge_subgraph.apply_edges(dgl.function.u_dot_v('x', 'x', 'score'), etype='u-r')
            return edge_subgraph.edata['score'][('user', 'u-r', 'recipe')].squeeze()


class RelationAttention(nn.Module):
    def __init__(self, in_size, hidden_size=16):
        super(RelationAttention, self).__init__()

        self.project = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False)
        )

    def forward(self, z):
        w = self.project(z).mean(0)                    # (M, 1)
        beta = torch.softmax(w, dim=0)                 # (M, 1)
        beta = beta.expand((z.shape[0],) + beta.shape) # (N, M, 1)
        out = (beta * z).sum(1)                        # (N, D * K)
        return out

def node_drop(feats, drop_rate, training):
    n = feats.shape[0]
    drop_rates = torch.FloatTensor(np.ones(n) * drop_rate)

    if training:
        masks = torch.bernoulli(1. - drop_rates).unsqueeze(1)
        feats = masks.to(feats.device) * feats / (1. - drop_rate)
    else:
        feats = feats
    return feats


class custom_GATConv(nn.Module):
    def __init__(self,
                 in_feats,
                 out_feats,
                 num_heads,
                 feat_drop=0.1,
                 attn_drop=0.,
                 negative_slope=0.2,
                 edge_drop=0.1,
                 residual=False,
                 activation=None,
                 allow_zero_in_degree=False,
                 bias=True):
        super(custom_GATConv, self).__init__()
        self._num_heads = num_heads
        self._in_src_feats, self._in_dst_feats = expand_as_pair(in_feats)
        self._out_feats = out_feats
        self._allow_zero_in_degree = allow_zero_in_degree
        if isinstance(in_feats, tuple):
            self.fc_src = nn.Linear(
                self._in_src_feats, out_feats * num_heads, bias=False)
            self.fc_dst = nn.Linear(
                self._in_dst_feats, out_feats * num_heads, bias=False)
            self.fc_src2 = nn.Linear(
                self._in_src_feats, out_feats * num_heads, bias=False)
            self.fc_dst2 = nn.Linear(
                self._in_dst_feats, out_feats * num_heads, bias=False)
        else:
            self.fc = nn.Linear(
                self._in_src_feats, out_feats * num_heads, bias=False)
            self.fc2 = nn.Linear(
                self._in_src_feats, out_feats * num_heads, bias=False)
        self.attn_l = nn.Parameter(torch.FloatTensor(size=(1, num_heads, out_feats)))
        self.attn_r = nn.Parameter(torch.FloatTensor(size=(1, num_heads, out_feats)))
        self.attn_l2 = nn.Parameter(torch.FloatTensor(size=(1, num_heads, out_feats)))
        self.attn_r2 = nn.Parameter(torch.FloatTensor(size=(1, num_heads, out_feats)))
        self.feat_drop = nn.Dropout(feat_drop)
        self.attn_drop = nn.Dropout(attn_drop)
        self.leaky_relu = nn.LeakyReLU(negative_slope)
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(size=(num_heads * out_feats,)))
        else:
            self.register_buffer('bias', None)
        if residual:
            if self._in_dst_feats != out_feats:
                self.res_fc = nn.Linear(
                    self._in_dst_feats, num_heads * out_feats, bias=False)
            else:
                self.res_fc = Identity()
        else:
            self.register_buffer('res_fc', None)
        self.reset_parameters()
        self.activation = activation

        self.edge_drop = edge_drop

    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        if hasattr(self, 'fc'):
            nn.init.xavier_normal_(self.fc.weight, gain=gain)
        else:
            nn.init.xavier_normal_(self.fc_src.weight, gain=gain)
            nn.init.xavier_normal_(self.fc_dst.weight, gain=gain)
        nn.init.xavier_normal_(self.attn_l, gain=gain)
        nn.init.xavier_normal_(self.attn_r, gain=gain)
        nn.init.xavier_normal_(self.attn_l2, gain=gain)
        nn.init.xavier_normal_(self.attn_r2, gain=gain)
        nn.init.constant_(self.bias, 0)
        if isinstance(self.res_fc, nn.Linear):
            nn.init.xavier_normal_(self.res_fc.weight, gain=gain)

    def set_allow_zero_in_degree(self, set_value):
        self._allow_zero_in_degree = set_value

    def forward(self, graph, feat, get_attention=False):
        with graph.local_scope():
            if not self._allow_zero_in_degree:
                if (graph.in_degrees() == 0).any():
                    raise DGLError('There are 0-in-degree nodes in the graph, '
                                   'output for those nodes will be invalid. '
                                   'This is harmful for some applications, '
                                   'causing silent performance regression. '
                                   'Adding self-loop on the input graph by '
                                   'calling `g = dgl.add_self_loop(g)` will resolve '
                                   'the issue. Setting ``allow_zero_in_degree`` '
                                   'to be `True` when constructing this module will '
                                   'suppress the check and let the code run.')

            if isinstance(feat, tuple):
                do_edge_drop = feat[2]
                # print('do_edge_drop: ', do_edge_drop)
                h_src = self.feat_drop(feat[0])
                h_dst = self.feat_drop(feat[1])
                h_src2 = h_src.clone()
                h_dst2 = h_dst.clone()
                if not hasattr(self, 'fc_src'):
                    feat_src = self.fc(h_src).view(-1, self._num_heads, self._out_feats)
                    feat_dst = self.fc(h_dst).view(-1, self._num_heads, self._out_feats)
                    feat_src2 = self.fc2(h_src2).view(-1, self._num_heads, self._out_feats)
                    feat_dst2 = self.fc2(h_dst2).view(-1, self._num_heads, self._out_feats)
                else:
                    feat_src = self.fc_src(h_src).view(-1, self._num_heads, self._out_feats)
                    feat_dst = self.fc_dst(h_dst).view(-1, self._num_heads, self._out_feats)
                    feat_src2 = self.fc_src2(h_src2).view(-1, self._num_heads, self._out_feats)
                    feat_dst2 = self.fc_dst2(h_dst2).view(-1, self._num_heads, self._out_feats)
            else:
                h_src = h_dst = self.feat_drop(feat)
                h_src2 = h_dst2 = h_src.clone() # self.feat_drop(feat)
                feat_src = feat_dst = self.fc(h_src).view(
                    -1, self._num_heads, self._out_feats)
                feat_src2 = feat_dst2 = self.fc(h_src).view(
                    -1, self._num_heads, self._out_feats)
                if graph.is_block:
                    feat_dst = feat_src[:graph.number_of_dst_nodes()]
                    feat_dst2 = feat_src2[:graph.number_of_dst_nodes()]

            el = (feat_src * self.attn_l).sum(dim=-1).unsqueeze(-1)
            er = (feat_dst * self.attn_r).sum(dim=-1).unsqueeze(-1)

            graph.srcdata.update({'ft': feat_src, 'el': el, 'feat_src2': feat_src2})
            graph.dstdata.update({'er': er, 'feat_dst2': feat_dst2})
            # compute edge attention, el and er are a_l Wh_i and a_r Wh_j respectively.
            graph.apply_edges(fn.u_add_v('el', 'er', 'e'))
            e = self.leaky_relu(graph.edata.pop('e'))

            # compute softmax, edge dropout
            if self.training and do_edge_drop and self.edge_drop > 0:
                perm = torch.randperm(graph.number_of_edges(), device=e.device)
                bound = int(graph.number_of_edges() * self.edge_drop)
                eids = perm[bound:]
                graph.edata["a"] = torch.zeros_like(e)
                graph.edata["a"][eids] = self.attn_drop(edge_softmax(graph, e[eids], eids=eids))
            else:
                graph.edata['a'] = self.attn_drop(edge_softmax(graph, e))

            # message passing
            graph.update_all(fn.u_mul_e('ft', 'a', 'm'),
                             fn.sum('m', 'initial_ft'))
            graph.update_all(fn.u_mul_v('feat_src2', 'feat_dst2', 'm2'),
                             fn.sum('m2', 'add_ft'))
            rst = graph.dstdata['initial_ft'] + graph.dstdata['add_ft']

            # residual
            if self.res_fc is not None:
                resval = self.res_fc(h_dst).view(h_dst.shape[0], self._num_heads, self._out_feats)
                rst = rst + resval
            # bias
            if self.bias is not None:
                rst = rst + self.bias.view(1, self._num_heads, self._out_feats)
            # activation
            if self.activation:
                rst = self.activation(rst)

            if get_attention:
                return rst, graph.edata['a']
            else:
                return rst


class GNN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.num_heads = 4 # 8
        self.hid_feats = int(hid_feats/self.num_heads)
        self.out_feats = int(out_feats/self.num_heads)
        self.relation_attention = RelationAttention(hid_feats)

        self.gatconv1 = custom_HeteroGraphConv({ # dglnn.HeteroGraphConv
            'i-r': custom_GATConv(in_feats, self.hid_feats, num_heads=self.num_heads),
            'r-i': custom_GATConv(in_feats, self.hid_feats, num_heads=self.num_heads),
            'r-r': custom_GATConv(in_feats, self.hid_feats, num_heads=self.num_heads),
            'i-i': custom_GATConv(in_feats, self.hid_feats, num_heads=self.num_heads),
            'u-r': custom_GATConv(in_feats, self.hid_feats, num_heads=self.num_heads),
            'r-u': custom_GATConv(in_feats, self.hid_feats, num_heads=self.num_heads),
            }, aggregate='stack')

        self.gatconv2 = custom_HeteroGraphConv({ # dglnn.HeteroGraphConv
            'i-r': custom_GATConv(self.hid_feats*self.num_heads, self.out_feats, num_heads=self.num_heads),
            'r-i': custom_GATConv(self.hid_feats*self.num_heads, self.out_feats, num_heads=self.num_heads),
            'r-r': custom_GATConv(self.hid_feats*self.num_heads, self.out_feats, num_heads=self.num_heads),
            'i-i': custom_GATConv(self.hid_feats*self.num_heads, self.out_feats, num_heads=self.num_heads),
            'u-r': custom_GATConv(self.hid_feats*self.num_heads, self.out_feats, num_heads=self.num_heads),
            'r-u': custom_GATConv(self.hid_feats*self.num_heads, self.out_feats, num_heads=self.num_heads),
            }, aggregate='stack')

        self.dropout = nn.Dropout(0.1)


    def forward(self, blocks, inputs, do_edge_drop):
        edge_weight_0 = blocks[0].edata['weight']
        edge_weight_1 = blocks[1].edata['weight']

        num_users = blocks[-1].dstdata[dgl.NID]['user'].shape[0]
        num_recipes = blocks[-1].dstdata[dgl.NID]['recipe'].shape[0]

        h = self.gatconv1(blocks[0], inputs, edge_weight_0, do_edge_drop)
        h = {k: F.relu(v).flatten(2) for k, v in h.items()}
        h = {k: self.relation_attention(v) for k, v in h.items()}

        first_layer_output = {}
        first_layer_output['user'] = h['user'][:num_users]
        first_layer_output['recipe'] = h['recipe'][:num_recipes]

        h = {key: self.dropout(value) for key, value in h.items()}
        h = self.gatconv2(blocks[-1], h, edge_weight_1, do_edge_drop)
        last_ingre_and_instr = h['recipe'].flatten(2)
        h = {k: self.relation_attention(v.flatten(2)) for k, v in h.items()}

        return h

#         # combine several layer embs as the final emb
#         combined_output = {}
#         combined_output['user'] = torch.cat([h['user'], first_layer_output['user']], dim=1)
#         combined_output['recipe'] = torch.cat([h['recipe'], first_layer_output['recipe']], dim=1)
#         combined_output['user'] = torch.add(h['user'], first_layer_output['user'])
#         combined_output['recipe'] = torch.add(h['recipe'], first_layer_output['recipe'])
#         return combined_output

## Losses and helper functions

In [ ]:
def norm(input, p=1, dim=1, eps=1e-12):
    return input / input.norm(p, dim, keepdim=True).clamp(min=eps).expand_as(input)

def get_recommendation_loss(pos_score, neg_score):
    n = pos_score.shape[0]
    return (neg_score.view(n, -1) - pos_score.view(n, -1) + 1).clamp(min=0).mean()

def get_contrastive_loss(x1, x2):
    temperature = 0.07

    # users
    x1_user, x2_user = F.normalize(x1['user']), F.normalize(x2['user'])
    pos_score_user = torch.mul(x1_user, x2_user).sum(dim=1)
    pos_score_user = torch.exp(pos_score_user/temperature)

    x2_user_neg = torch.flipud(x2_user)
    ttl_score_user = torch.mul(x1_user, x2_user_neg).sum(dim=1)
    ttl_score_user = pos_score_user + torch.exp(ttl_score_user/temperature)

    contrastive_loss_user = - torch.log(pos_score_user/ttl_score_user).mean()
    # print('contrastive_loss_user: ', contrastive_loss_user)
    assert not math.isnan(contrastive_loss_user)


    # recipes
    x1_recipe, x2_recipe = F.normalize(x1['recipe']), F.normalize(x2['recipe'])
    pos_score_recipe = torch.mul(x1_recipe, x2_recipe).sum(dim=1)
    pos_score_recipe = torch.exp(pos_score_recipe/temperature)

    x2_recipe_neg = torch.flipud(x2_recipe)
    ttl_score_recipe = torch.mul(x1_recipe, x2_recipe_neg).sum(dim=1)
    ttl_score_recipe = pos_score_recipe + torch.exp(ttl_score_recipe/temperature) #.sum(dim=1)

    contrastive_loss_recipe = - torch.log(pos_score_recipe/ttl_score_recipe).mean()
    # print('contrastive_loss_recipe: ', contrastive_loss_recipe)

    return contrastive_loss_user + contrastive_loss_recipe

def get_emb_loss(*params):
    out = None
    for param in params:
        for k,v in param.items():
            if out == None:
                out = (v**2/2).mean()
            else:
                out += (v**2/2).mean()
    return out

## Model

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.user_embedding = nn.Sequential(
            nn.Linear(300, 128),
            nn.ReLU(),
        )
        self.instr_embedding = nn.Sequential(
            nn.Linear(1024, 128),
            nn.ReLU(),
        )
        self.ingredient_embedding = nn.Sequential(
            nn.Linear(46, 128),
            nn.ReLU()
        )
        self.recipe_combine2out = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU()
        )
        self.gnn = GNN(128, 128, 128, graph.etypes)
        self.pred = ScorePredictor()
        self.setTransformer_ = SetTransformer()

    def forward(self, positive_graph, negative_graph, blocks, input_features):
        user, instr, ingredient, ingredient_of_dst_recipe = input_features

        # major GNN
        user_major = self.user_embedding(user)
        user_major = norm(user_major)
        instr_major = self.instr_embedding(instr)
        instr_major = norm(instr_major)
        ingredient_major = self.ingredient_embedding(ingredient)
        ingredient_major = norm(ingredient_major)
        x = self.gnn(blocks, {'user': user_major, 'recipe': instr_major, 'ingredient': ingredient_major}, torch.Tensor([[0]]))

        # contrastive - 1
        user1 = node_drop(user, 0.1, model.training)
        instr1 = node_drop(instr, 0.1, model.training)
        ingredient1 = node_drop(ingredient, 0.1, model.training)

        user1 = self.user_embedding(user1)
        user1 = norm(user1)
        instr1 = self.instr_embedding(instr1)
        instr1 = norm(instr1)
        ingredient1 = self.ingredient_embedding(ingredient1)
        ingredient1 = norm(ingredient1)

        x1 = self.gnn(blocks, {'user': user1, 'recipe': instr1, 'ingredient': ingredient1}, torch.Tensor([[1]]))

        # contrastive - 2
        user2 = node_drop(user, 0.1, model.training)
        instr2 = node_drop(instr, 0.1, model.training)
        ingredient2 = node_drop(ingredient, 0.1, model.training)

        user2 = self.user_embedding(user2)
        user2 = norm(user2)
        instr2 = self.instr_embedding(instr2)
        instr2 = norm(instr2)
        ingredient2 = self.ingredient_embedding(ingredient2)
        ingredient2 = norm(ingredient2)

        x2 = self.gnn(blocks, {'user': user2, 'recipe': instr2, 'ingredient': ingredient2}, torch.Tensor([[1]]))

        # setTransformer
        all_ingre_emb_for_each_recipe = get_ingredient_neighbors_all_embeddings(blocks, blocks[1].dstdata['_ID']['recipe'], ingredient_of_dst_recipe)
        all_ingre_emb_for_each_recipe = norm(all_ingre_emb_for_each_recipe)
        total_ingre_emb = self.setTransformer_(all_ingre_emb_for_each_recipe) # 1
        total_ingre_emb = norm(total_ingre_emb)

        # scores
        x['recipe'] = self.recipe_combine2out(total_ingre_emb.add(x['recipe']))
        pos_score = self.pred(positive_graph, x)
        neg_score = self.pred(negative_graph, x)

        return pos_score, neg_score, x1, x2

## Metrics and evaluation methods

In [ ]:
def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).cpu().numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).cpu().numpy()
    return roc_auc_score(labels, scores)

def precision_at_k(r, k):
    # Relevance is binary (nonzero is relevant).
    assert k >= 1
    r = np.asarray(r)[:k]
    return np.mean(r)

def recall_at_k(r, k, all_pos_num):
    r = np.asfarray(r)[:k]
    return np.sum(r) / all_pos_num

def dcg_at_k(r, k, method=0):
    # method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
    #         If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max


def average_precision_at_k(r, Ks):
    r = np.asarray(r) != 0
    out = []
    for k in Ks:
        assert k <= len(r)
        # print('[precision_at_k(r, i + 1) for i in range(k) if r[i]]: ', [precision_at_k(r, i + 1) for i in range(k) if r[i]])
        all_precision_before_k = [precision_at_k(r, i + 1) for i in range(k) if r[i]]
        if len(all_precision_before_k) == 0:
            all_precision_before_k = [0]
        out.append(np.mean(all_precision_before_k))
    if not out:
        return 0.
    # return np.array([np.mean(out)])
    return np.array(out)

def get_map_at_k(rs, Ks):
    # examples:
    # average_precision_at_k([1, 1, 0, 1, 0, 1, 0, 0, 0, 1], [5,10])
    # average_precision_at_k([0, 0, 1, 0, 0, 0, 0, 0, 0, 1], [5,10])
    # get_map_at_k([[1, 1, 0, 1, 0, 1, 0, 0, 0, 1,1], [0, 0, 1, 0, 0, 0, 0, 0, 0, 1,1]], [5,10])
    out = np.zeros(len(Ks))
    for r in rs:
        # print('average_precision_at_k: ', average_precision_at_k(r, Ks))
        out += average_precision_at_k(r, Ks)/len(rs)
    return out
    # return np.mean([average_precision(r) for r in rs])

def get_ranklist_for_one_user(user_poss, user_negs, Ks):
    item_scores = {}
    n_pos = len(user_poss)
    n_neg = len(user_negs)
    for i in range(n_pos):
        item_scores[i] = user_poss[i]
    for i in range(n_neg):
        item_scores[i+1] = user_negs[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_scores, key=item_scores.get)

    r = []
    for i in K_max_item_score:
        if i < n_pos:
            r.append(1)
        else:
            r.append(0)
    return r

def hit_at_k(r, k):
    r = np.array(r)[:k]
    if np.sum(r) > 0:
        return 1.
    else:
        return 0.

def get_mrr(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])


def get_performance_one_user(user_poss, user_negs, Ks):
    r = get_ranklist_for_one_user(user_poss, user_negs, Ks)

    precision, recall, ndcg, hit_ratio = [], [], [], []
    for K in Ks:
        precision.append(precision_at_k(r, K))
        # recall.append(recall_at_k(r, K, len(user_poss)))
        ndcg.append(ndcg_at_k(r, K))
        hit_ratio.append(hit_at_k(r, K))
    # return {'precision': np.array(precision), 'recall': np.array(recall),
    #         'ndcg': np.array(ndcg), 'hit_ratio': np.array(hit_ratio)}, r
    return {'precision': np.array(precision),
            'ndcg': np.array(ndcg), 'hit_ratio': np.array(hit_ratio)}, r


def get_performance_all_users(user2pos_score_dict, user2neg_score_dict, Ks):
    # all_result = {'precision': np.zeros(len(Ks)), 'recall': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)),
    #           'hit_ratio': np.zeros(len(Ks))}
    all_result = {'hit_ratio': np.zeros(len(Ks)), 'ndcg': np.zeros(len(Ks)), 'precision': np.zeros(len(Ks))}

    rs = []
    n_test_users = len(user2pos_score_dict)

    # one specific user
    for user in user2pos_score_dict.keys():
        user_pos_score = user2pos_score_dict[user]
        user_neg_score = user2neg_score_dict[user]
        one_result, one_r = get_performance_one_user(user_pos_score, user_neg_score, Ks)
        # all_result['recall'] += one_result['recall']/n_test_users
        all_result['hit_ratio'] += one_result['hit_ratio']/n_test_users
        all_result['ndcg'] += one_result['ndcg']/n_test_users
        all_result['precision'] += one_result['precision']/n_test_users
        rs.append(one_r)

    # get MRR
    # MRR = get_mrr(rs)
    # all_result['MRR'] = MRR

    # get MAP
    MAP = get_map_at_k(rs, Ks)
    all_result['MAP'] = MAP

    return all_result


def evaluate(model, dataloader, multi_metrics=False):
    # print('start evaluating ...')
    evaluate_start = time.time()
    model.eval()
    total_loss = 0
    epoch_contrastive_loss = 0
    iteration_cnt = 0
    total_pos_score = torch.tensor([]).to(device)
    total_neg_score = torch.tensor([]).to(device)

    # for evaluation
    user2pos_score_dict = {}
    user2neg_score_dict = {}

    with torch.no_grad():
        for input_nodes, positive_graph, negative_graph, blocks in dataloader:
            blocks = [b.to(device) for b in blocks]
            positive_graph = positive_graph.to(device)
            negative_graph = negative_graph.to(device)

            input_user = blocks[0].srcdata['random_feature']['user']
            input_instr = blocks[0].srcdata['avg_instr_feature']['recipe']
            input_ingredient = blocks[0].srcdata['nutrient_feature']['ingredient']
            ingredient_of_dst_recipe = blocks[1].srcdata['nutrient_feature']['ingredient']
            input_features = [input_user, input_instr, input_ingredient, ingredient_of_dst_recipe]

            pos_score, neg_score, x1, x2 = model(positive_graph, negative_graph, blocks, input_features)
            contrastive_loss = get_contrastive_loss(x1, x2)
            total_pos_score = torch.cat([total_pos_score, pos_score])
            total_neg_score = torch.cat([total_neg_score, neg_score])

            recommendation_loss = get_recommendation_loss(pos_score, neg_score)
            loss = recommendation_loss # + 0.01 * contrastive_loss
            total_loss += recommendation_loss.item()
            epoch_contrastive_loss += contrastive_loss.item()
            iteration_cnt += 1

            # for evaluation
            global_test_users = blocks[1].dstdata['_ID']['user'] # we need to map the user id in subgraph to the whole graph
            test_users, test_recipes = positive_graph.edges(etype='u-r')
            test_users = test_users.tolist()
            test_recipes = test_recipes.tolist()
            for index in range(len(test_users)):
                test_u = int(global_test_users[test_users[index]])
                test_r = int(test_recipes[index])
                test_score = float(pos_score[index])

                if test_u not in user2pos_score_dict:
                    user2pos_score_dict[test_u] = []
                user2pos_score_dict[test_u].append(test_score)

                if test_u not in user2neg_score_dict:
                    user2neg_score_dict[test_u] = neg_score[index*n_test_negs:(index+1)*n_test_negs]

            # break

        total_loss /= iteration_cnt
        epoch_contrastive_loss /= iteration_cnt

        # metrics
        auc = compute_auc(total_pos_score, total_neg_score)
        if multi_metrics:
            # evaluation_result = get_performance_all_users(total_pos_score, total_neg_score, Ks)
            evaluation_result = get_performance_all_users(user2pos_score_dict, user2neg_score_dict, Ks)
            evaluation_result['AUC'] = auc
            print('evaluation_result: ', evaluation_result)
            print('epoch_contrastive_loss: ', epoch_contrastive_loss)
        else:
            print('AUC: ', auc)

        evalutate_time = time.strftime("%M:%S min", time.gmtime(time.time()-evaluate_start))
        print('evalutate_time: ', evalutate_time)
    return total_loss

## Training

In [ ]:
model = Model().to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.9)

print('start ... ')
for epoch in range(50):
    train_start = time.time()
    epoch_loss = 0
    epoch_contrastive_loss = 0
    epoch_emb_loss = 0
    iteration_cnt = 0

    for input_nodes, positive_graph, negative_graph, blocks in train_edgeloader:
        model.train()
        blocks = [b.to(device) for b in blocks]
        positive_graph = positive_graph.to(device)
        negative_graph = negative_graph.to(device)

        input_user = blocks[0].srcdata['random_feature']['user']
        input_instr = blocks[0].srcdata['avg_instr_feature']['recipe']
        input_ingredient = blocks[0].srcdata['nutrient_feature']['ingredient']
        ingredient_of_dst_recipe = blocks[1].srcdata['nutrient_feature']['ingredient']
        input_features = [input_user, input_instr, input_ingredient, ingredient_of_dst_recipe]

        pos_score, neg_score, x1, x2 = model(positive_graph, negative_graph, blocks, input_features)
        contrastive_loss = get_contrastive_loss(x1, x2)
        # emb_loss = get_emb_loss(x1, x2)
        assert not math.isnan(contrastive_loss)
        recommendation_loss = get_recommendation_loss(pos_score, neg_score)
        assert not math.isnan(recommendation_loss)

        loss = recommendation_loss + 0.1 * contrastive_loss # + 1e-5 * emb_loss
        opt.zero_grad()
        loss.backward()
        opt.step()

        epoch_loss += recommendation_loss.item()
        epoch_contrastive_loss += contrastive_loss.item()
        # epoch_emb_loss += emb_loss.item()
        iteration_cnt += 1

        # break

    epoch_loss /= iteration_cnt
    epoch_contrastive_loss /= iteration_cnt
    train_end = time.strftime("%M:%S min", time.gmtime(time.time()-train_start))

    print('Epoch: {0},  Loss: {l:.4f}, Contrastive: {cl:.4f}, Emb: {el:.4f},  Time: {t}, LR: {lr:.6f}'
          .format(epoch, l=epoch_loss, cl=epoch_contrastive_loss, el=epoch_emb_loss, t=train_end, lr=opt.param_groups[0]['lr']))
    scheduler.step()

    # Evaluation
    # For demonstration purpose, only test set result is reported here. Please use val_dataloader for comprehensiveness.
    if epoch >= 4 and epoch % 1 == 0:
        print('testing: ')
        evaluate(model, test_edgeloader, multi_metrics=True)
        print()
        print()

start ... 
Epoch: 0,  Loss: 0.6383, Contrastive: 0.2247, Emb: 0.0000,  Time: 03:51 min, LR: 0.005000
Epoch: 1,  Loss: 0.4799, Contrastive: 0.0617, Emb: 0.0000,  Time: 03:52 min, LR: 0.004500
Epoch: 2,  Loss: 0.4433, Contrastive: 0.0460, Emb: 0.0000,  Time: 03:50 min, LR: 0.004050
Epoch: 3,  Loss: 0.4208, Contrastive: 0.0361, Emb: 0.0000,  Time: 03:49 min, LR: 0.003645
Epoch: 4,  Loss: 0.4016, Contrastive: 0.0289, Emb: 0.0000,  Time: 03:49 min, LR: 0.003281
testing: 
evaluation_result:  {'hit_ratio': array([0.17728358, 0.22238975, 0.25731876, 0.28558864, 0.31385852,
       0.34062068, 0.36348788, 0.386732  , 0.40909662, 0.42982787]), 'ndcg': array([0.17728358, 0.22238975, 0.2444275 , 0.25856244, 0.27073762,
       0.28109063, 0.28923609, 0.29698413, 0.30403939, 0.31028011]), 'precision': array([0.17728358, 0.11119487, 0.08577292, 0.07139716, 0.0627717 ,
       0.05677011, 0.05192684, 0.0483415 , 0.04545518, 0.04298279]), 'MAP': array([0.17728358, 0.19983666, 0.21147967, 0.21854714, 0.22

KeyboardInterrupt: 